In [ ]:
import cv2
import numpy as np

Download sample files (if run this notebook on Google Colab)

In [ ]:
!curl "https://gau-nernst.github.io/MOOC/OpenCV-Python-tutorials/Lenna_(test_image).png" -o "Lenna_(test_image).png"
!curl "https://gau-nernst.github.io/MOOC/OpenCV-Python-tutorials/tesla_coil.mp4" -o "tesla_coil.mp4"

# Read image from file

In [ ]:
img_path = "Lenna_(test_image).png"
img = cv2.imread(img_path)

cv2.imshow("Image from file", img)  # Display image
cv2.waitKey(0)                      # Wait forever until a key is pressed
cv2.destroyAllWindows()             # Close all windows, MUST use when run in Jupyter Notebook

# Read video from file

In [ ]:
video_path = "tesla_coil.mp4"
cap = cv2.VideoCapture(video_path)

while True:
    success, img = cap.read()
    
    # success is None when no more frame (video ends)
    if success:
        cv2.imshow("Video from file", img)
        # cv2.waitKey(1) will wait for 1ms before moving on
        # Press 'q' to stop playback
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

# Read video from webcam

In [ ]:
cap = cv2.VideoCapture(0)

# Set VideoCaptureProperties
# Width and Height
cap.set(3, 640)
cap.set(4, 480)
# See full list here: https://docs.opencv.org/master/d4/d15/group__videoio__flags__base.html#gaeb8dd9c89c10a5c63c139bf7c4f5704d

while True:
    success, img = cap.read()
    cv2.imshow("Video from webcam", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

# Some Open-CV functions on images

In [ ]:
img_path = "Lenna_(test_image).png"
img = cv2.imread(img_path)

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur filter
blur = cv2.GaussianBlur(img, (13,13), 0)

# Stack grayscale image 3 times to create a 3-channel image
gray3 = np.stack((gray,gray,gray), axis=2)
# Stack images horizontally
output = np.hstack((img, gray3, blur))
cv2.imshow('Images', output)

cv2.waitKey(0)
cv2.destroyAllWindows()

# Resize and crop

In [ ]:
img_path = "Lenna_(test_image).png"
img = cv2.imread(img_path)
cv2.imshow("Original", img)

# Resize (width, height)
img = cv2.resize(img, (400,500))
cv2.imshow("Resized", img)

print('Array format (HWC): ', img.shape)

# Crop [Height, Width, Channel] → (HWC) format
img = img[200:400, 100:500]
cv2.imshow("Cropped", img)

cv2.waitKey(0)
cv2.destroyAllWindows()

# Draw shapes and text

In [ ]:
img = np.zeros((512,512,3), np.uint8)

cv2.line(img, (0,0), (300,300), (255, 0, 0), 2)
cv2.rectangle(img, (100,100), (200,200), (0, 255, 0), 1)
cv2.putText(img, "Open-CV", (300,200), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)

cv2.imshow("Shapes", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Create a mask from color range

In [ ]:
img_path = "Lenna_(test_image).png"
img = cv2.imread(img_path)

# Convert to HSV color space
# H = [0,179], S = [0,255], V = [0,255]
imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

lower = np.array([90,120,0])
upper = np.array([160,255,255])

# cv2.inRange() will return a 1-channel image, with pixel values either 0 or 255
mask = cv2.inRange(imgHSV, lower, upper)

# Apply mask
img_masked = cv2.bitwise_and(img, img, mask=mask)

mask3 = np.stack((mask, mask, mask), axis=2)
output = np.hstack((img, mask3, img_masked))
cv2.imshow("Masking", output)

cv2.waitKey(0)
cv2.destroyAllWindows()

# Simple Object detection

In [ ]:
img_path = "Lenna_(test_image).png"
img = cv2.imread(img_path)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Load pre-trained Cascade Classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Perform face detection
faces = face_cascade.detectMultiScale(gray)
for (x,y,w,h) in faces:
    # Draw bounding box
    img = cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)

cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Slider control

In [ ]:
# Masking code
img_path = "Lenna_(test_image).png"
img = cv2.imread(img_path)
imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
lower = np.array([90,0,0])
upper = np.array([160,255,255])
HUE_RANGE = 80

def update_mask(hue=0):
    lower[0] = np.max((hue - HUE_RANGE // 2,   0))
    upper[0] = np.min((hue + HUE_RANGE // 2, 179))
    
    mask = cv2.inRange(imgHSV, lower, upper)
    img_masked = cv2.bitwise_and(img, img, mask=mask)
    mask3 = np.stack((mask, mask, mask), axis=2)
    output = np.hstack((img, mask3, img_masked))
    cv2.imshow("Masking", output)
update_mask()

# Add a window named 'Control'
cv2.namedWindow('Control')
# Create a Trackbar named 'Hue' with callback update_mask
cv2.createTrackbar('Hue', 'Control', 0, 179, update_mask)

cv2.waitKey(0)
cv2.destroyAllWindows()